In [20]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import countDistinct
import math

In [21]:
spark = SparkSession.builder.getOrCreate()
df  = spark.read\
        .option('header', True)\
        .csv('./202203-divvy-tripdata/202203-divvy-tripdata.csv')

#df_rdd = spark.sparkContext.parallelize(df)
#df_rdd.getNumPartitions()
df.show(10)

+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+------------------+------------------+--------------+--------------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|         start_lat|         start_lng|       end_lat|       end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+------------------+------------------+--------------+--------------+-------------+
|47EC0A7F82E65D52| classic_bike|2022-03-21 13:45:01|2022-03-21 13:51:18|Wabash Ave & Wack...|    TA1307000131|Kingsbury St & Ki...|  KA1503000043|         41.886875|         -87.62603|41.88917683258|-87.6385057718|       member|
|8494861979B0F477|electric_bike|2022-03-16 09:37:16|2022-03-16 09:43:34|Michigan Ave

In [3]:
type_ride = df.select('rideable_type').distinct().rdd.map(lambda x: x[0]).collect()

n_type_ride = df.select(countDistinct('rideable_type'))
n_type_ride.show()

type_ride

+-----------------------------+
|count(DISTINCT rideable_type)|
+-----------------------------+
|                            3|
+-----------------------------+



['docked_bike', 'electric_bike', 'classic_bike']

In [4]:
member = df.filter(df.member_casual == 'member')
casual = df.filter(df.member_casual != 'member')

In [11]:
member.select('rideable_type')\
        .groupBy('rideable_type')\
        .count()\
        .show(5, truncate=False)

casual.select('rideable_type')\
        .groupBy('rideable_type')\
        .count()\
        .show(5, truncate=False)

+-------------+-----+
|rideable_type|count|
+-------------+-----+
|electric_bike|95108|
|classic_bike |99052|
+-------------+-----+

+-------------+-----+
|rideable_type|count|
+-------------+-----+
|docked_bike  |8358 |
|electric_bike|46137|
|classic_bike |35387|
+-------------+-----+



In [19]:
member = member.withColumn('lat_diff', member.start_lat-member.end_lat)
member = member.withColumn('long_diff', member.start_lat-member.end_lat)

member.show(5)

+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+-----------------+------------------+--------------+--------------+-------------+--------------------+--------------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|        start_lat|         start_lng|       end_lat|       end_lng|member_casual|            lat_diff|           long_diff|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+-----------------+------------------+--------------+--------------+-------------+--------------------+--------------------+
|47EC0A7F82E65D52| classic_bike|2022-03-21 13:45:01|2022-03-21 13:51:18|Wabash Ave & Wack...|    TA1307000131|Kingsbury St & Ki...|  KA1503000043|        41.886875|         -87.62603|41.8891

In [ ]:
def degs_to_km(diff_lat, diff_long, lat):
    km_v = abs(diff_lat) * 110.574
    km_h = abs(diff_long) * 111.320 * math.cos(lat)
    return km_v, km_h

def euclidean_distance(func):
    return math.sqrt(func[0]**2 + func[1]**2)